In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed
from keras.layers import Concatenate, Flatten
from keras.layers import GRU, Conv2D, MaxPooling2D
from keras.layers import Input, Reshape, Dot, Add
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import RMSprop
# from keras.utils.vis_utils import plot_model
import keras
import keras.backend as K
from data_handle import *
from gensim_wrapper import *
from utils import *
import gensim
import random
import numpy as np
import tensorflow as tf
seed_val = 1000
random.seed(seed_val)
np.random.seed(seed_val)
tf.set_random_seed(seed_val)

In [ ]:
def evaluate(final_embedding, word2int, embed_size):
    gensim = GensimWrapper('data/news.txt',embed_size, 0, log=False)
    gensim.set_embeddings(word2int, final_embedding)
    result = gensim.evaluate()
    for key in result:
        print("{0}: {1:.2f}%".format(key, result[key]), end=' ')
    print()

In [ ]:
words = read_file()
unkown_word = "<unk>"
# words = [unkown_word] + words
xvocab, xword2int, xint2word = build_vocab(words)

words, word2freq = min_count_threshold(words)
vocab, word2int, int2word = build_vocab(words)
print(len(vocab))
# word2freq = get_frequency(words, word2int, int2word)
unigrams = [word2freq[int2word[i]] for i in range(len(word2int))]

char2int, int2char, char2tup, tup2char, n_consonant, n_vowel = build_charset()
ns_unigrams = ns_sample(word2freq, word2int, int2word, .75)
n_chars = 11 + 2 
n_features = len(char2int)
batch_size = 120
embed_size = 100
skip_window = 5

In [ ]:
def parseVec(file, delimiter):
    lines = open(file, encoding='utf8').readlines()
    vocab_size, embed_size = [int(s) for s in lines[0].split()]
    embeddings = np.ndarray((vocab_size, embed_size), dtype=np.float32)
    for i in range(vocab_size):
        line = lines[i+1].split(delimiter)[:-1]
        word = line[0]
        if word in word2int:
            wordvec = np.array([float(j) for j in line[1:]])
            embeddings[word2int[word]] = wordvec
    return embeddings

In [ ]:
util = Utils(word2int, int2word, fast)
print(util.evaluate_word_analogy('data/analogies.txt'))

In [ ]:
utils = Utils(embedding=fast, word2int=word2int, int2word=int2char)
utils.sorted_sim("ኢትዮጵያ")

In [ ]:
evaluate(embed_size=embed_size,final_embedding=normalize(fast), word2int=word2int)
evaluate(embed_size=embed_size,final_embedding=fast, word2int=word2int)

In [ ]:
utils = Utils(embedding=cew, word2int=word2int, int2word=int2char)
utils.sorted_sim("ኢትዮጵያ")

In [ ]:
cew = parseVec('results/w2v.txt', ' ')
# print(cew.shape)
# cew = np.load('results/wi.npy')
# cew = np.concatenate([cew, np.zeros((1, 100))], axis=0)
print(cew.shape)

In [ ]:
evaluate(embed_size=cew.shape[1],final_embedding=normalize(cew), word2int=word2int) 
evaluate(embed_size=cew.shape[1],final_embedding=cew, word2int=word2int)